In [23]:
import random
import numpy as np
import unittest


class Carte(object):
    #la carte possede une valeur et une couleur(symbole)
    def __init__(self,valeur,couleur):
        self.__valeur = valeur
        self.__couleur = couleur

    #recupération des donnees privees
    def get_valeur(self):
        return self.__valeur
    def get_couleur(self):
        return self.__couleur
    
    #methode speciale de python qui permet d'afficher directement la carte
    def __str__(self):
       return f"{self.__valeur} de {self.__couleur}"


class JeuDeCartes(object):
    #toutes les cartes de 2 a 10 puis valet, dame, roi et as(plus forte)
    def __init__(self):
        self.__cartes = [Carte(valeur, couleur) for valeur in range(2, 15) for couleur in ['Coeur', 'Carreau', 'Trèfle', 'Pique']]
        random.shuffle(self.__cartes) #melange les cartes

    #distribution
    def donner_cartes(self):
        return self.__cartes[:26], self.__cartes[26:]
        

In [24]:
class TestJeuDeCartes(unittest.TestCase):
    def test_carte(self):
        carte = Carte(5, 'Coeur')
        self.assertEqual(carte.get_valeur(), 5)
        self.assertEqual(carte.get_couleur(), 'Coeur')
    
    def test__jeudecartes(self):
        jeu = JeuDeCartes()
        self.assertEqual(len(jeu.donner_cartes()[0]), 26)
        self.assertEqual(len(jeu.donner_cartes()[1]), 26)

In [25]:
class Joueur(object):
    def __init__(self,nom):
        self.__nom = nom
        self.__defausse = []
        self.__cartes = []

    #recuperation des attributs prives
    def get_nom(self):
        return self.__nom
    def get_defausse(self):
        return self.__defausse
    def get_cartes(self):
        return self.__cartes
    
    #piocher dans les cartes disponible ou dans la defausse quand il n'a plus de cartes
    def piocher(self):
        if not self.__cartes: #si pas de cartes alors il melange la defausse et l'utilise
            random.shuffle(self.__defausse)
            self.__cartes = self.__defausse
            self.__defausse = []
        return self.__cartes.pop() #retire et retourne la derniere carte

    #modifie directement la defausse avec les deux cartes jouées
    def ajouter_defausse(self, *cartes):
        self.__defausse.extend(cartes)

    #modifie derectement les cartes du joueur
    def ajouter_pioche(self, *cartes):
        self.__cartes.extend(cartes)


In [26]:
class TestJeuDeCartes(unittest.TestCase):
    def test_joueur(self):
        joueur = Joueur("Test")
        self.assertEqual(joueur.get_nom(), "Test")
        self.assertEqual(len(joueur.get_cartes()), 0)
        self.assertEqual(len(joueur.get_defausse()), 0)

    def test_piocher(self):
        joueur = Joueur("Test")
        joueur.ajouter_pioche(Carte(5, 'Carreau'))
        self.assertEqual(len(joueur.get_cartes()), 1)
        joueur.piocher()
        self.assertEqual(len(joueur.get_cartes()), 0)

    def test_ajouter_defausse(self):
        joueur = Joueur("Test")
        joueur.ajouter_defausse(Carte(7, 'Trèfle'))
        self.assertEqual(len(joueur.get_defausse()), 1)

    def test_ajouter_pioche(self):
        joueur = Joueur("Test")
        joueur.ajouter_pioche(Carte(8, 'Pique'))
        self.assertEqual(len(joueur.get_cartes()), 1)

In [27]:
#deroulement de la bataille
def bataille(joueur1, joueur2):
    #cartes en jeu pendant la bataille (celles retournees)
    cartes_en_jeu = []
    #chaque joueur pioche
    carte1 = joueur1.piocher()
    carte2 = joueur2.piocher()
    cartes_en_jeu.extend([carte1, carte2]) #mise a jour des cartes en jeu
    
    print(f"{carte1} vs {carte2}")

    #BATAILLE si la valeur des cartes en jeu est égale
    while carte1.get_valeur() == carte2.get_valeur():
        print("-> Bataille")
        #les joueurs piochent une carte mais ne la retourne pas
        cartes_cachees = [joueur1.piocher(), joueur2.piocher()]
        cartes_en_jeu.extend(cartes_cachees)
        print(f"[cartes cachées] {cartes_cachees[0]} et {cartes_cachees[1]}")
        #les joueurs retourne une cartes
        carte1 = joueur1.piocher()
        carte2 = joueur2.piocher()
        cartes_en_jeu.extend([carte1, carte2])

        print(f"{carte1} vs {carte2}")

    #sinon, comparasion entre la valeur des cartes en jeu : le plus grand gagne
    if carte1.get_valeur() > carte2.get_valeur():
        print(f"-> Gagnant : {joueur1.get_nom()} {len(joueur1.get_cartes()) + len(cartes_en_jeu)}/{len(joueur2.get_cartes())}")
        joueur1.ajouter_pioche(*cartes_en_jeu)
    else:
        print(f"-> Gagnant : {joueur2.get_nom()} {len(joueur1.get_cartes())}/{len(joueur2.get_cartes()) + len(cartes_en_jeu)}")
        joueur2.ajouter_pioche(*cartes_en_jeu)


In [28]:
#Execution du jeu
jeu = JeuDeCartes()
joueur1 = Joueur("j1")
joueur2 = Joueur("j2")
#chaque joueur possede 26 cartes
joueur1.ajouter_pioche(*jeu.donner_cartes()[0])
joueur2.ajouter_pioche(*jeu.donner_cartes()[1])

#la jeu continue tant qu'aucun joueur n'a plus de cartes
while joueur1.get_cartes() and joueur2.get_cartes():
    bataille(joueur1, joueur2)

#affichage du gagnant final !
if not joueur1.get_cartes():
    print(f"---> Gagnant final : {joueur2.get_nom()} BRAVO!")
else:
    print(f"---> Gagnant final : {joueur1.get_nom()} BRAVO!")


5 de Carreau vs 4 de Pique
-> Gagnant : j1 27/25
4 de Pique vs 8 de Carreau
-> Gagnant : j2 26/26
5 de Carreau vs 8 de Carreau
-> Gagnant : j2 25/27
4 de Coeur vs 8 de Carreau
-> Gagnant : j2 24/28
2 de Pique vs 8 de Carreau
-> Gagnant : j2 23/29
11 de Coeur vs 8 de Carreau
-> Gagnant : j1 24/28
8 de Carreau vs 2 de Pique
-> Gagnant : j1 25/27
2 de Pique vs 4 de Coeur
-> Gagnant : j2 24/28
8 de Carreau vs 4 de Coeur
-> Gagnant : j1 25/27
4 de Coeur vs 2 de Pique
-> Gagnant : j1 26/26
2 de Pique vs 5 de Carreau
-> Gagnant : j2 25/27
4 de Coeur vs 5 de Carreau
-> Gagnant : j2 24/28
8 de Carreau vs 5 de Carreau
-> Gagnant : j1 25/27
5 de Carreau vs 4 de Coeur
-> Gagnant : j1 26/26
4 de Coeur vs 2 de Pique
-> Gagnant : j1 27/25
2 de Pique vs 4 de Pique
-> Gagnant : j2 26/26
4 de Coeur vs 4 de Pique
-> Bataille
[cartes cachées] 5 de Carreau et 2 de Pique
8 de Carreau vs 10 de Trèfle
-> Gagnant : j2 23/29
11 de Coeur vs 10 de Trèfle
-> Gagnant : j1 24/28
10 de Trèfle vs 8 de Carreau
-> Gagna